Fetching content from the database

In [1]:
import sqlite3
import pandas as pd

# Connect to the database
conn = sqlite3.connect('imdb_data.db')
cursor = conn.cursor()

# Converting data to a dataframe
movies = pd.read_sql_query("SELECT * FROM movies",conn)

print(movies.head())
conn.close()

   id                        title  year duration    rating  IMDB rating
0   1  1. The Shawshank Redemption  1994   2h 22m         R          9.3
1   2             2. The Godfather  1972   2h 55m         R          9.2
2   3           3. The Dark Knight  2008   2h 32m     PG-13          9.0
3   4     4. The Godfather Part II  1974   3h 22m         R          9.0
4   5              5. 12 Angry Men  1957   1h 36m  Approved          9.0


In [2]:
print(movies.dtypes)

id               int64
title           object
year             int64
duration        object
rating          object
IMDB rating    float64
dtype: object


I already inspected the full SQL table and noticed the ratings of the movies aren't quite right. So I would leave these out for now.
First of all, I would like to extract the ranking number from the movie names and create a new column "ranking". I think I'd also like to convert the duration to minutes, so I will create another column "duration_minutes".

In [6]:
movies['ranking'] = movies['title'].str.split('.').str[0].astype(int)
movies['title'] = movies['title'].str.split('.').str[1].str.strip()
print(movies.head())

   id                     title  year duration    rating  IMDB rating  ranking
0   1  The Shawshank Redemption  1994   2h 22m         R          9.3        1
1   2             The Godfather  1972   2h 55m         R          9.2        2
2   3           The Dark Knight  2008   2h 32m     PG-13          9.0        3
3   4     The Godfather Part II  1974   3h 22m         R          9.0        4
4   5              12 Angry Men  1957   1h 36m  Approved          9.0        5


In [7]:
def convert_to_minutes(duration):
    hours = int(duration.split('h')[0]) if 'h' in duration else 0
    minutes = int(duration.split('h')[-1].split('m')[0].strip()) if 'm' in duration else 0
    return (hours * 60) + minutes

movies['duration_minutes'] = movies['duration'].apply(convert_to_minutes)
print(movies.head())


   id                     title  year duration    rating  IMDB rating  \
0   1  The Shawshank Redemption  1994   2h 22m         R          9.3   
1   2             The Godfather  1972   2h 55m         R          9.2   
2   3           The Dark Knight  2008   2h 32m     PG-13          9.0   
3   4     The Godfather Part II  1974   3h 22m         R          9.0   
4   5              12 Angry Men  1957   1h 36m  Approved          9.0   

   ranking  duration_minutes  
0        1               142  
1        2               175  
2        3               152  
3        4               202  
4        5                96  


In [20]:
print(movies.dtypes)

id                    int64
title                object
year                  int64
duration             object
rating               object
IMDB rating         float64
ranking               int32
duration_minutes      int64
dtype: object


What's the longest movie in the list? What's the shortest? What's the average duration?
Creating a new sorted dataframe, where I've sorted the movies based on duration (descending).

In [29]:
import numpy as np

# The longest movie
idx_longest = movies['duration_minutes'].idxmax()
longest_hours = int(movies['duration_minutes'].max()/60)
longest_remaining_minutes = movies['duration_minutes'].max()%60
title_longest = movies.loc[idx_longest,'title']

print(f"The longest movie in the list is \"{title_longest}\" with a duration of {longest_hours} hours and {longest_remaining_minutes} minutes.")

#The shortest movie
idx_shortest = movies['duration_minutes'].idxmin()
shortest_duration = movies['duration_minutes'].min()
title_shortest = movies.loc[idx_shortest,'title']

print(f"The shortest movie in the list is \"{title_shortest}\" with a duration of {shortest_duration} minutes.")

#The average duration
avg_duration = int(movies['duration_minutes'].mean())
avg_duration_hours = int(avg_duration/60)
avg_duration_minutes = int(avg_duration%60)
print(f"The average duration for movies in Top250 list is {avg_duration_hours} hours and {avg_duration_minutes} minutes.")

#Sorted dataframe based on duration + writing the dataframe to a csv
movies_sorted_by_duration = movies.sort_values(by='duration_minutes',ascending=False)
#print(movies_sorted_by_duration.head())
movies_sorted_by_duration.to_csv('movies_sorted_by_duration.csv',index=False)

The longest movie in the list is "Gone with the Wind" with a duration of 3 hours and 58 minutes.
The shortest movie in the list is "Sherlock Jr" with a duration of 45 minutes.
The average duration for movies in Top250 list is 2 hours and 9 minutes.
